# Homework 3 of ys3031, EECS 6894

In [ ]:
print("hlwd")

# Problem 1

# Problem 2

In [ ]:
import keras 
model = keras.applications.resnet50.ResNet50(include_top=True,
                                            weights='imagenet',
                                            input_tensor=None,
                                            input_shape=None,
                                            pooling=None,
                                            classes=1000)

In [ ]:
model.summary()

In [ ]:
conv1=9472+256
conv2a=4160+256+36928+256+16640+16640+1024+1024
conv2b=16448+256+36928+256+16640+1024 
conv2c= 16448+256+36928+256+16640+1024
conv3a= 32896+512+147584+512+66048+131584+2048+2048
conv3b= 65664+512+147584+512+66048+2048
conv3c= 65664+512+147584+512+66048+2048                                                                 
conv3d= 65664+512+147584+512+66048+2048
conv4a= 131328+1024+590080+1024+263168+525312+4096+4096
conv4b= 262400+1024+590080+1024+263168+4096
conv4c= 262400+1024+590080+1024+263168+4096                                                                 
conv4d= 262400+1024+590080+1024+263168+4096
conv4e= 262400+1024+590080+1024+263168+4096
conv4f= 262400+1024+590080+1024+263168+4096
conv5a= 524800+2048+2359808+2048+1050624+2099200+8192+8192                                                                 
conv5b= 1049088+2048+2359808+2048+1050624+8192                                                                 
conv5c= 1049088+2048+2359808+2048+1050624+8192                                                                 
convfc= 2049000

conv2 = conv2a+conv2b+conv2c
conv3 = conv3a+conv3b+conv3c+conv3c
conv4 = conv4a+conv4b+conv4c+conv4d+conv4e+conv4f
conv5 = conv5a+conv5b+conv5c

print("conv1:", conv1)
print("conv2:", conv2)
print("conv3:", conv3)
print("conv4:", conv4)
print("conv5:", conv5)

print(conv1+conv2a+conv2b+conv2c+conv3a+conv3b+conv3c+conv3d+conv4a+conv4b+conv4c+conv4d+conv4e+conv4f+conv5a+conv5b+conv5c+convfc)

# Problem 3

In [ ]:
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications.vgg16 import VGG16
import os

def data_aug(category, starname):
    for filename in os.listdir(os.path.join("data_origin", category, starname)):
        if filename.endswith(".jpg"):
            filenameandpath = os.path.join("data_origin", category, starname, filename)
            print(filenameandpath)
            img = load_img(os.path.join("data_origin", category, starname, filename))
            img_array = img_to_array(img)
            img_array = img_array.reshape((1,) + img_array.shape)
            i = 0
            datagen = ImageDataGenerator(
                            rotation_range=40,
                            width_shift_range=0.2,
                            height_shift_range=0.2,
                            shear_range=0.2,
                            zoom_range=0.2,
                            horizontal_flip=True,
                            fill_mode='nearest')
            for _ in datagen.flow(img_array, 
                                  batch_size=1, 
                                  save_to_dir=os.path.join("data_expanded", category, starname), 
                                  save_prefix=starname, 
                                  save_format="jpeg"):
                i += 1
                if i > 20:
                    break
            # for pic in os.listdir(os.path.join("data_expanded", category, starname)):
            #     if pic.endswith(".jpeg"):
            #         expanded_img_array = vgg_pre.predict(img_to_array(load_img(os.path.join("data_exapnded", category, 
            #                                                                 starname, pic), target_size=(224,224,3))))
                
                
                    
data_aug("train", "jennifer")
data_aug("validation", "jennifer")
data_aug("train", "justin")
data_aug("validation", "justin")

data_origin/train/jennifer/1-FaceId-0.jpg
data_origin/train/jennifer/6-FaceId-0.jpg
data_origin/train/jennifer/7-FaceId-0.jpg


In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras.applications.vgg16 import VGG16


# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data_expanded/train'
validation_data_dir = 'data_expanded/validation'
nb_train_samples = 2
nb_validation_samples = 80
epochs = 10
batch_size = 5

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
vgg_pre = VGG16(include_top=False, input_shape=input_shape)

top_model_pre = Sequential()

top_model_pre.add(Flatten(input_shape=input_shape))
top_model_pre.add(Dense(64))
top_model_pre.add(Activation('relu'))
top_model_pre.add(Dropout(0.5))
top_model_pre.add(Dense(1))
top_model_pre.add(Activation('sigmoid'))

model_pre = Sequential()
# model_pre.add(vgg_pre)
model_pre.add(top_model_pre)

for layer in model_pre.layers[:19]:
    layer.trainable = False

model_pre.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model_pre.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

top_model_pre.save_weights('top_model_pre_for_fine_tune.h5')

Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.
Epoch 1/10


AttributeError: 'ProgbarLogger' object has no attribute 'log_values'

In [ ]:

'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# path to the model weights files.
top_model_weights_path = 'pre_trained_for_fine_tune.h5'
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 10
batch_size = 16

# build the VGG16 network
vgg_model = VGG16(include_top=False, input_shape=(img_width,img_height,3))
print('Model loaded.')
# build a classifier model to put on top of the convolutional model
top_model = Sequential()

top_model.add(Flatten(input_shape=vgg_model.output_shape[1:]))
top_model.add(Dense(256))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.8))
top_model.add(Dense(64))
top_model.add(Activation('relu'))
top_model.add(Dropout(0.8))
top_model.add(Dense(1))
top_model.add(Activation('sigmoid'))

# print(top_model.output_shape)

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
model = Sequential()
model.add(vgg_model)
model.add(top_model)

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:28]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples,
    epochs=epochs,
    validation_data=validation_generator,
    nb_val_samples=nb_validation_samples)

In [ ]:
img_selfie = load_img('./selfie/sunyiming.jpeg', target_size=(150, 150))
selfie_array = img_to_array(img_selfie)
selfie_array = selfie_array.reshape((1,) + selfie_array.shape)
selfie_array /= 255.0
result = model.predict_classes(selfie_array)

In [ ]:
result

# Problem 4

In [ ]:
je_pic = load_img('./selfie/jennifer.jpeg', target_size=(224, 224))
je_array = img_to_array(je_pic)
print(je_array.shape)

In [ ]:
import foolbox
import keras
import numpy as np
from keras.applications.resnet50 import ResNet50

# instantiate model
keras.backend.set_learning_phase(0)
kmodel = ResNet50(weights='imagenet')
preprocessing = (np.array([104, 116, 123]), 1)
fmodel = foolbox.models.KerasModel(kmodel, bounds=(0, 255), preprocessing=preprocessing)

# get source image and label
image, label = foolbox.utils.imagenet_example()
print(image.shape, type(image))
print(je_array.shape, type(image))

# apply attack on source image
# ::-1 reverses the color channels, because Keras ResNet50 expects BGR instead of RGB
attack = foolbox.attacks.FGSM(fmodel)
adversarial = attack(je_array[:, :, ::-1], label)

In [ ]:
import os 
print(os.getcwd())